#### **Import Libraries**

In [1]:
import anthropic
import os 
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from tqdm import tqdm
import numpy as np 
import random 
import math
from itertools import chain
from IPython.display import display, Markdown
import textwrap
import tiktoken
import csv
import time 
from trics.nlp.utils import to_markdown, create_csv_with_headers

#### **Setup Folder**

In [4]:
version = 1 
experiment_folder = './../../../toy-data/exp2/'
data_csv = experiment_folder + f'data_{version}.csv'


#### **Plotting**

In [6]:
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import rcParams
rcParams['image.interpolation'] = 'nearest'
rcParams['image.cmap'] = 'viridis'
rcParams['axes.grid'] = False
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use('seaborn-v0_8-dark-palette')

from matplotlib import font_manager 
locations = './../../../styles/Newsreader'
font_files = font_manager.findSystemFonts(fontpaths=locations)
print(locations)
print(font_files[0])
for f in font_files: 
    font_manager.fontManager.addfont(f)
plt.rcParams["font.family"] = "Newsreader"

./../../../styles/Newsreader
/home/ubuntu/llmft/styles/Newsreader/static/Newsreader_60pt/Newsreader_60pt-ExtraLight.ttf


#### **LLM Set Up**

In [7]:
client = anthropic.Anthropic(
    api_key=###
)


#### **Define File**

In [8]:
if os.path.exists(data_csv):
    print(f'Data for {version} already exists')
    run = False 
else:
    create_csv_with_headers(data_csv, ['Race', 'Gender', 'Rent', 'Health', 'Fault', 'Description'])
    run = True 

File not found. Creating new file with headers: ['Race', 'Gender', 'Rent', 'Health', 'Fault', 'Description']


#### **Functions**

In [9]:

def anthropic_completion(input):
    message = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=500,
    temperature=1.,
    system="You are a housing court lawyer",
    messages=[
        {"role": "user", "content": input}
    ])
    return message.content[0].text

def get_prompt(i, x):
    return f"""random seed: i
    Task: Write a brief description of an {'African American' if x[0] == 1 else 'Non-African American'} {'Female' if x[1] == 1 else 'Non-Female'} tenant who is currently behind on rent for ${x[2]}.
    Mention that they have relatively {'poor health' if x[3] == 1 else 'good health'}, and also include detailts that indicate that they {'are at fault for their eviction' if x[4] == 1 else 'are not at fault for their eviction.'}
    
    Description:  The tenant is a """

#### **Create Variables**

In [10]:
# Set number of observations
n = 500 

# Set the random seed for reproducibility
np.random.seed(2)

# Generate a single Bernoulli random variable with p = 0.5
race = np.random.binomial(n=1, p=0.5, size=n)
gender = np.random.binomial(n=1, p=0.5, size=n)
rent = np.random.normal(loc=1000, scale=200, size=n)
health = np.random.binomial(n=1, p=0.5, size=n)
fault = np.random.binomial(n=1, p=0.5, size=n)
data_matrix = np.column_stack((race, gender, rent, health, fault))
print(data_matrix.shape)

(500, 5)


In [11]:
for i in range(n):
    print(f"Creating a description of tenant {i}")
    text = anthropic_completion(get_prompt(i, data_matrix[i]))
    with open(data_csv, mode='a', newline='') as file:
        writer = csv.writer(file)
        row = list(data_matrix[i]) + [text]
        writer.writerow(row)
    time.sleep(0.1)

Creating a description of tenant 0
Creating a description of tenant 1
Creating a description of tenant 2
Creating a description of tenant 3
Creating a description of tenant 4
Creating a description of tenant 5
Creating a description of tenant 6
Creating a description of tenant 7
Creating a description of tenant 8
Creating a description of tenant 9
Creating a description of tenant 10
Creating a description of tenant 11
Creating a description of tenant 12
Creating a description of tenant 13
Creating a description of tenant 14
Creating a description of tenant 15
Creating a description of tenant 16
Creating a description of tenant 17
Creating a description of tenant 18
Creating a description of tenant 19
Creating a description of tenant 20
Creating a description of tenant 21
Creating a description of tenant 22
Creating a description of tenant 23
Creating a description of tenant 24
Creating a description of tenant 25
Creating a description of tenant 26
Creating a description of tenant 27
Cr